# Stochastic Gradient Descent

In [1]:
%load_ext autoreload
%autoreload 2

# Global imports

In [2]:
import sys

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

# Local imports

In [3]:
sys.path.append('..')

from base_code.models.poc import NeuralNetwork as NN
from base_code.datasets import DryBeansDataset as DBD
from base_code.train_test import train, test

from base_code.preprocessing.nomalize_standarizer import pipeline as normalize_standarize
from base_code.dataloaders.base import ContinualLearningDataLoader as DataLoader

# Data preprocessing

In [4]:
dataset = DBD()

In [5]:
dataset.get_task_range(0, 2).labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [6]:
pipeline = normalize_standarize(dataset.features)

In [7]:
X = pipeline.transform(dataset.features)
dataset.features = X

# DataLoader settings

In [8]:
train_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Neural Netwrok

In [9]:
model = NN(features_shape=dataset.features_shape, output_shape=dataset.output_shape)

In [10]:
loss_fn = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train - Test

## Concurrent

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, train_dataloader, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.956650  [   10/13611]
loss: 1.960915  [ 1010/13611]
loss: 1.942756  [ 2010/13611]
loss: 1.937229  [ 3010/13611]
loss: 1.939797  [ 4010/13611]
loss: 1.935474  [ 5010/13611]
loss: 1.904520  [ 6010/13611]
loss: 1.936473  [ 7010/13611]
loss: 1.887173  [ 8010/13611]
loss: 1.890128  [ 9010/13611]
loss: 1.871152  [10010/13611]
loss: 1.741034  [11010/13611]
loss: 1.920559  [12010/13611]
loss: 1.717723  [13010/13611]
Test Error: 
 Accuracy: 34.2%, Avg loss: 1.783969 

Epoch 2
-------------------------------
loss: 1.869239  [   10/13611]
loss: 1.672736  [ 1010/13611]
loss: 1.741121  [ 2010/13611]
loss: 1.715134  [ 3010/13611]
loss: 1.930799  [ 4010/13611]
loss: 1.858136  [ 5010/13611]
loss: 1.805385  [ 6010/13611]
loss: 1.682133  [ 7010/13611]
loss: 1.813624  [ 8010/13611]
loss: 1.528674  [ 9010/13611]
loss: 1.757904  [10010/13611]
loss: 1.672263  [11010/13611]
loss: 1.558595  [12010/13611]
loss: 1.707351  [13010/13611]
Test Error: 
 Accuracy: 61.5

## Sequential

In [12]:
epochs = 5
for task_id in range(train_dataloader.get_tasks_length()):
    print(f"Task {task_id+1}\n-------------------------------")
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(model, train_dataloader.get_task(task_id), loss_fn, optimizer)
        test(model, train_dataloader.get_task_range(0, task_id+1), loss_fn)

Task 1
-------------------------------
Epoch 1
-------------------------------
loss: 1.276212  [   10/ 2027]
loss: 1.297114  [ 1010/ 2027]
loss: 1.187896  [ 2010/ 2027]
Test Error: 
 Accuracy: 98.7%, Avg loss: 1.201651 

Epoch 2
-------------------------------
loss: 1.204704  [   10/ 2027]
loss: 1.175839  [ 1010/ 2027]
loss: 1.219845  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.4%, Avg loss: 1.185685 

Epoch 3
-------------------------------
loss: 1.166602  [   10/ 2027]
loss: 1.170341  [ 1010/ 2027]
loss: 1.174511  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.8%, Avg loss: 1.178281 

Epoch 4
-------------------------------
loss: 1.179792  [   10/ 2027]
loss: 1.168353  [ 1010/ 2027]
loss: 1.172774  [ 2010/ 2027]
Test Error: 
 Accuracy: 99.9%, Avg loss: 1.174184 

Epoch 5
-------------------------------
loss: 1.165649  [   10/ 2027]
loss: 1.165880  [ 1010/ 2027]
loss: 1.173934  [ 2010/ 2027]
Test Error: 
 Accuracy: 100.0%, Avg loss: 1.171874 

Task 2
-------------------------------
Epoch 1
---